# req

> Fill in a module description here

In [ ]:
#| default_exp req

In [ ]:
#| export
import requests
from pathlib import Path
from html2text import HTML2Text  
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import traceback
from scraper.core import *
from lxml.html import fromstring, HtmlElement, tostring
from lxml_html_clean import Cleaner
from collections import deque
import sys
from fastcore.all import *
from time import time

h2t = HTML2Text(bodywidth=1000_000)
h2t.ignore_links = True
h2t.mark_code = True
h2t.ignore_images = True
cleaner = Cleaner(javascript=True, style=True)

In [ ]:
#| export

class Req:
    def __init__(self, url):
        self.url = url
        self.fn = get_fn_from_url(url)
        self.loc_domain = urlparse(self.url).netloc
        self.dir = None

    def html2md(self):
        try:
            resp = requests.get(self.url, timeout=100)
            resp.raise_for_status()
            soup = BeautifulSoup(resp.text, 'html.parser')

            body = fromstring(str(soup)).xpath('//body')
        
            if not body:
                raise ValueError("No <body> tag found in HTML.")
            
            body = cleaner.clean_html(body[0])

            assert isinstance(body, HtmlElement)
            """
            print([a.get('href') for a in body.xpath('//a[@href]')])
            print([valid_href(a.get('href')) for a in body.xpath('//a[@href]')])

            print(self.loc_domain)
            print([hydrate_links(self.loc_domain, a.get('href')) 
                                            for a in body.xpath('//a[@href]') 
                                             ])
            print(f"{self.fn.split('.html')[0]}.md")
            
            """
            with open(self.dir/f"{self.fn.split('.html')[0]}.md", 'w', encoding='utf-8') as f:
                    f.write(h2t.handle(tostring(body, encoding='unicode')))
            
            self.links = set([link for link in [hydrate_links(self.loc_domain, a.get('href')) 
                                            for a in body.xpath('//a[@href]') 
                                            if  valid_href(a.get('href')) ] if link])
        except Exception as e:
            traceback.print_exception(type(e), e, e.__traceback__)
    
    def download_file(self):
        """
        Downloads a file from the specified URL and saves it to the given path
        """
        try:
            resp = requests.get(self.url)
            if resp.status_code == 200:
                with open(self.dir/self.fn, 'wb') as f:
                    f.write(resp.content)
        except Exception as e:
            traceback.print_exception(type(e), e, e.__traceback__)
    
    def __call__(self, dir_n:Path):
        try:
            self.dir = dir_n/self.loc_domain
            self.dir.mkdir(parents=True, exist_ok=True)
            f_ext = '.'+self.fn.split('.')[-1]
            
            if f_ext == '.html':
                self.html2md()
            elif f_ext in ['.pdf', '.doc', '.docx', '.odt', '.xls', '.xlsx', '.ppt', '.pptx', '.txt', '.csv']:
                self.download_file()
            else:
                print(f"could not process {self.url}")
        except Exception as e:
            traceback.print_exception(type(e), e, e.__traceback__)

In [ ]:
#| hide
md_fn = Path('Test/fastcore.fast.ai/_.md')
md_fn.unlink(missing_ok=True)

In [ ]:
req = Req('https://fastcore.fast.ai/')
req(Path('Test/'))
print(md_fn)
assert md_fn.exists(), "File not found"
assert md_fn.stat().st_size > 0, "File is empty"

Test/fastcore.fast.ai/_.md


In [ ]:
#| hide
pdf_fn = Path('Test/www.w3.org/_wai_er_tests_xhtml_testfiles_resources_pdf_dummy.pdf')
pdf_fn.unlink(missing_ok=True)

In [ ]:
req = Req('https://www.w3.org/WAI/ER/tests/xhtml/testfiles/resources/pdf/dummy.pdf')
req(Path('Test/'))
print(pdf_fn)
assert pdf_fn.exists(), "File not found"
assert pdf_fn.stat().st_size > 0, "File is empty"

Test/www.w3.org/_wai_er_tests_xhtml_testfiles_resources_pdf_dummy.pdf


In [ ]:
#| hide
md_fn.unlink(missing_ok=True)
pdf_fn.unlink(missing_ok=True)

In [ ]:
#| export
class spider:
    """
    A simple web crawler (spider).
    The `spider` class is responsible for visiting URLs, processing them to extract 
    links, and then continuing the crawl recursively. It keeps track of visited 
    URLs to avoid revisiting the same page.

    Attributes:
    -----------
    url : str
        The starting URL for the crawl.
    dir : Path
        The directory where downloaded content will be saved (default: 'Test/').
    to_visit : deque
        A deque (queue) of URLs that need to be visited, initialized with the starting URL.
    visited : set
        A set of URLs that have already been processed to avoid reprocessing.

    Methods:
    --------
    show(url, links_sz):
        Prints the current status of the crawl, including the URL being processed, 
        the number of links found, and the sizes of the `visited` and `to_visit` lists.

    process_url(url):
        Processes a single URL: fetches content, extracts links, and adds new links 
        to the queue (`to_visit`).

    run():
        The main method that runs the spider. It processes URLs in `to_visit` using 
        the `process_url` method. It processes URLs one by one, and extracts and 
        follows links found on each page.
    """
    def __init__(self, url:str, dir_n:Path=Path('Test/')):
        self.url = url
        self.dir = dir_n

        self.to_visit = deque([url])
        self.visited = set()
    
    def show(self, url, links_sz):
        sys.stdout.write(
            f"\rProcessed: {url[:50]}... | Links found: {links_sz:<10} | "
            f"Visited: {len(self.visited):<10} | To visit: {len(self.to_visit):<10} "
        )
        sys.stdout.flush()
        
    def process_url(self, url: str):
        """Process a single URL: fetch content, extract links, and add to queue."""
        try:
            if url not in self.visited:
                self.visited.add(url)  # Mark as visited

                # Request and process the page
                req = Req(url)
                req(self.dir)
                links = getattr(req, "links", [])
                self.to_visit.extend(links)
                self.show(url, len(links))
        except Exception as e:
            print(f"Error processing {url}")
            traceback.print_exc()
            
    def run(self):
        """Main method to run the spider, processing URLs with parallel threads"""
        start = time()
        while self.to_visit:
            # Get URLs to process in the current batch
            self.process_url(self.to_visit.pop())
        
        print(f"\ntime taken: {(time()-start):.2} sec")

In [ ]:
#| hide
%rm -rf Test/

In [ ]:
s = spider('https://fastcore.fast.ai/', Path('Test/'))
s.run()

Processed: https://fastcore.fast.ai/docments.html... | Links found: 18         | Visited: 23         | To visit: 168          90        
time taken: 1.4e+01 sec


In [ ]:
#| hide
assert len(Path('Test/fastcore.fast.ai').ls()) <= len(s.visited)
assert len(s.to_visit) == 0

FileNotFoundError: [Errno 2] No such file or directory: 'Test/fastcore.fast.ai'

In [ ]:
#| hide
%rm -rf Test/
import nbdev; nbdev.nbdev_export()